In [31]:
import urllib
import json
import sys
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

In [ ]:
filename = sys.argv[1]
location = urllib.parse.quote(input("Put the place that you want to collect: "))
restaurants = get_store(location)
data = get_review_set(restaurants)


In [32]:
def get_store(location):
    '''
    input: host_url ("https://www.mangoplate.com"), location ("대전 유성구 어은동", this should be converted into unicode)
    output: list containing review site (['https://www.mangoplate.com/restaurants/JVom8cKaKsBo', ... ])
    '''
    host_url = "https://www.mangoplate.com"
    # location = "%EB%8C%80%EC%A0%84%20%EC%9C%A0%EC%84%B1%EA%B5%AC%20%EC%96%B4%EC%9D%80%EB%8F%99"
    restaurant = []
    pageNum = 0
    while True:
        pageNum += 1
        url = host_url + "/search/" +  location + "?" + "keyword=" + location + "&" + "page=" + str(pageNum)
        r = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(r, "html.parser")        
        stores = json.loads(soup.find('script', type="application/json", id="search_first_result_json").text.strip().replace("&quot;",'"'))
        
        if (len(stores) == 0):
            break
        
        for s in stores:
            restaurant.append(s["restaurant"]["restaurant_uuid"])

    print("The number of stores:", len(restaurant))
    return restaurant

In [22]:
print(get_store("%EB%8C%80%EC%A0%84%20%EC%9C%A0%EC%84%B1%EA%B5%AC%20%EC%96%B4%EC%9D%80%EB%8F%99"))

The number of stores: 116
[280192, 189576, 233407, 250669, 109082, 244863, 219283, 37421, 37379, 188202, 192362, 254855, 194071, 188819, 37559, 37697, 193894, 231672, 194074, 193887, 37709, 37395, 37439, 191503, 37770, 325525, 277507, 165344, 273414, 277864, 232043, 193167, 213596, 203144, 272574, 37381, 193214, 223614, 272170, 37505, 203189, 37405, 222868, 37834, 193897, 272115, 193864, 194486, 185813, 274940, 282312, 193902, 194075, 193888, 193898, 165086, 284857, 188452, 193881, 194078, 193891, 194080, 193886, 193940, 194086, 194076, 271131, 193890, 193900, 195128, 37866, 264429, 260564, 37808, 193889, 271859, 270956, 276041, 256784, 271674, 273472, 271795, 236609, 260525, 223544, 37528, 193901, 193882, 193895, 194072, 190217, 194079, 193899, 165085, 193885, 193907, 193892, 193904, 193893, 193908, 193903, 193883, 193884, 202420, 37720, 165090, 37884, 193873, 165327, 37754, 165313, 208871, 164925, 276012, 324089, 328652]


In [33]:
def get_review_set(restaurants):
    member_uuid = []
    member_nick_name = []
    review_rating = []
    reg_time = []
    comment = []
    restaurant_uuid = []
    restaurant_name = []
    restaurant_address = []
    restaurant_lat = []
    restaurant_long = []
    
    for restaurant in restaurants:
    
        review_json_address = "https://stage.mangoplate.com/api/v5/restaurants/" + str(restaurant) + "/reviews.json?request_count=9999&start_index=0"

        # Test Address
        # review_json_address = 'https://stage.mangoplate.com/api/v5/restaurants/184801/reviews.json?access_token=&language=kor&request_count=99&start_index=0'
        
        review_json_data = json.loads(requests.get(review_json_address).text)
        for r in review_json_data:
                member_uuid.append(r["user"]["member_uuid"])
                member_nick_name.append(r["user"]["nick_name"])
                review_rating.append(r["action_value"])
                reg_time.append(r["time"])
                comment.append(r["comment"]["comment"])
                restaurant_uuid.append(r["restaurant"]["restaurant_uuid"])
                restaurant_name.append(r["restaurant"]["name"])
                restaurant_address.append(r["restaurant"]["address"])
                restaurant_lat.append(r["restaurant"]["latitude"])
                restaurant_long.append(r["restaurant"]["longitude"])
                
    return pd.DataFrame({'Restaurant ID': restaurant_uuid,
                         'Restaurant Name': restaurant_name,
                         'Restaurant Address': restaurant_address,
                         'Restaurant Latitude': restaurant_lat,
                         'Restaurant Longitude': restaurant_long,
                        'Member ID': member_uuid, 
                        'Member Nickname': member_nick_name,
                        'Rating': review_rating,
                        'Time': reg_time,
                        'Comment': comment})


In [34]:
test = get_review_set(get_store("%eb%8c%80%ec%a0%84%ec%8b%9c"))
test.to_pickle("대전시.pkl")
# test.to_csv('test.csv', sep='\t', index=False, encoding='utf-8')

The number of stores: 1000


In [36]:
test.to_csv("대전시.csv", sep='\t', index=False, encoding='utf-8')

In [35]:
dj = pd.read_pickle("대전시.pkl")
dj


,Comment,Member ID,Member Nickname,Rating,Restaurant Address,Restaurant ID,Restaurant Latitude,Restaurant Longitude,Restaurant Name,Time
0,1114밀 사장님께서 디저트 공부하셔서 오픈한 디저트샵이에요! 이전에도 틈틈히 팝...,47875,준영,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-02-08 22:24:37
1,오랜만에 몰아쓰는 모엘르 신상아가들ㅎㅎㅎㅎ♥\n 개강하고 그동안 꽤 많은 메뉴들이...,47875,준영,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-04-25 14:35:55
2,간만에 어은동을 왔더니 왜 이리 낯선 가게들이 많은 건지? 그래도 여기는 인터넷 보...,533317,푱,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-04-09 18:40:11
3,오픈한 지가 언제고 다녀온 지가 언젠데 ㅜㅜㅜ 이제야 적어봅니다....!\n먹어본 ...,641116,하영,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-03-10 18:51:20
4,네번째 가보는 모엘르(Moelleux)-말차쇼콜라무스와 말차타르트 \n\n늘 테잌아...,577319,별이,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-03-11 21:18:47
5,어제 새로운 디저트가 추가되었다고 해서 다녀왔어요! 저녁시간 쯤 출발하게 돼서 남아...,641116,하영,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-03-12 13:09:42
6,"대전에 위치한 디저트샵 ""모엘르"". 후기 보며 항상 가보고 싶었지만 갈 수 없는 곳...",33537,Jㅐ,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-03-15 20:01:30
7,모엘르(Moelleux)- 다섯번째 \n\n모엘르는 일요일과 월요일에 쉰다. 가장 ...,577319,별이,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-06-27 18:38:26
8,여기는 얘기도 많이 들어서 너무 오고 싶었던데인데 직접 와보니 의외로 크기가 작아서...,699254,유동현,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-04-22 22:14:24
9,모엘르(Moelleux) 세번째 - 바닐라타르트\n\n흰자태가 고와서 아껴두고 먹...,577319,별이,3,대전시 유성구 어은동 111-4,280192,36.3637504,127.35762,모엘르,2017-03-02 16:04:05
